In [ ]:
import delta, pyspark.sql.functions

In [ ]:
%run /utils/environment

In [ ]:
function = ''

In [ ]:
def CompressZbpc_c12():
    target_table = f'{raw_adls_path}/SAP/BW/ZBPC_C12'
    print(target_table)
    delta_df = delta.DeltaTable.forPath(spark, target_table).toDF().persist()
    print("Pre count:", delta_df.count())
    delta_df.printSchema()

    delta_df = delta_df.groupBy(['FISCPER3','FISCVARNT','FISCYEAR','FISCPER','CURKEY_GC','CURKEY_LC','UNIT','CURKEY_TC','SEM_CGCOMP','CO_AREA','FUNC_AREA','COUNTRY','MOVE_TYPE','ACQ_YEAR','ACQ_PER','SEM_ACOMP','BCS_APRCTR','CS_PLEVEL','BCS_DOCTY','BCS_CTFLG','BCS_INVCOM','BCS_INVCPC','PROFIT_CTR','PCOMP_CODE','SEM_CGPRCT','COMP_CODE','PART_PRCTR','VERSION','/BIC/ZCHR_ACT','/BIC/ZGL_ACCT','COMPANY','PCOMPANY','COSTCENTER','/BIC/ZAUDT_TRL','/BIC/ZELIM_FLG']).sum()\
        .withColumnsRenamed({'sum(CS_TRN_GC)':'CS_TRN_GC','sum(CS_TRN_LC)':'CS_TRN_LC','sum(CS_TRN_QTY)':'CS_TRN_QTY','sum(CS_TRN_TC)':'CS_TRN_TC','sum(/BIC/ZCS_GC_EU)':'/BIC/ZCS_GC_EU','sum(/BIC/ZCS_GC_GB)':'/BIC/ZCS_GC_GB'})\
        .withColumn('CS_TRN_GC',pyspark.sql.functions.col('CS_TRN_GC').cast('decimal(17,2)'))\
        .withColumn('CS_TRN_LC',pyspark.sql.functions.col('CS_TRN_LC').cast('decimal(17,2)'))\
        .withColumn('CS_TRN_QTY',pyspark.sql.functions.col('CS_TRN_QTY').cast('decimal(17,3)'))\
        .withColumn('CS_TRN_TC',pyspark.sql.functions.col('CS_TRN_TC').cast('decimal(17,2)'))\
        .withColumn('/BIC/ZCS_GC_EU',pyspark.sql.functions.col('/BIC/ZCS_GC_EU').cast('decimal(17,2)'))\
        .withColumn('/BIC/ZCS_GC_GB',pyspark.sql.functions.col('/BIC/ZCS_GC_GB').cast('decimal(17,2)'))
    print("Post count:", delta_df.count())
    delta_df.printSchema()

    delta_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").option("path", target_table).save()

In [ ]:
match function:
    case 'ZBPC_C12':
        CompressZbpc_c12()
    case _:
        notebookutils.mssparkutils.notebook.exit(0)

In [ ]:
notebookutils.mssparkutils.notebook.exit(0)

Repartition Delta Table

In [ ]:
deltaTable = delta.DeltaTable.forPath(spark, f"{raw_adls_path}/SAP/BW/0DELIV_NUMB$P")

In [ ]:
delta_df = deltaTable.toDF()
delta_df.count()

In [ ]:
#Uncomment to use
#delta_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true") \
#  .option("path", f"{raw_adls_path}/SAP/BW/0DELIV_NUMB_NEW$P") \
#  .partitionBy("CREATEDON").save()

Selective Delete

In [ ]:
deltaTable = delta.DeltaTable.forPath(spark, f"{raw_adls_path}/SAP/BW/ZBILLDOC")

In [ ]:
delta_detail_df = deltaTable.detail()
delta_detail_df.drop('name','description','minReaderVersion','minWriterVersion').show(vertical=True,truncate=False)

In [ ]:
#Uncomment to use
#deltaTable.delete("FISCYEAR IN ('2024','2023')")

Rollback version

In [ ]:
deltaTable = delta.DeltaTable.forPath(spark, f"{raw_adls_path}/SAP/BW/ZBILLDOC")

In [ ]:
delta_history_df = deltaTable.history(3)
delta_history_df.drop('userId','userName','job','notebook','clusterId','userMetadata').show(vertical=True,truncate=False)

In [ ]:
#In case of bad delete, reverse with above code based on version (get from history table above)
#deltaTable.restoreToVersion(#version number)

Read raw CDC delta and push to raw/bronze table (~2025, hasn't been used in years, does this still work?)

In [ ]:
#Read the staging CSV file
df1= spark.read.format('csv').load("abfss://staging@azwwwprodprdadapadls.dfs.core.windows.net/ZPUR202320230917/*",header='true',inferSchema='true')

#Remove extra CDC columns that don't show up in the dataflow view
df1=df1.drop('ODQ_CHANGEMODE','ODQ_ENTITYCNTR','_PACKAGEID','_SEQUENCENUMBER')

#Optionally add a DELTA_ID column
from datetime import datetime, timedelta, date
from pytz import timezone
now_utc = datetime.now().strftime("%Y%m%d%H%M%S")
print(not_utc)
#df1=df1.withColumn('DELTA_ID',F.lit(now_utc))

#Create as Table for SQL Insert later (gets around format issues easily)
df1.createOrReplaceTempView("TempTable1")

In [ ]:
#Read the target raw delta table
df=spark.read.format('delta').load("abfss://raw@azwwwprodprdadapadls.dfs.core.windows.net/SAP/BW/ZPURCHASE")

#Create as Table for SQL Insert later
df.createOrReplaceTempView("TempTable")

In [ ]:
%%sql
insert into TempTable 
select * from TempTable1
